In [1]:
#Imports
from scripts import bookdatafunctions as bdf
from scripts import corpusMLfunctions as cmf
import pandas as pd
import numpy as np
from datasets import Dataset, disable_progress_bars
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.metrics import f1_score
import optuna
import json
import multiprocessing as mp
import shutil
import warnings
from tqdm import tqdm
from pprint import pprint

#Constants
AGES = ['5','6','7','8','9','10','11','12','13','14','15']
KEYLISTS = "Keylists.jsonl"
keylists = []
with open(KEYLISTS, 'r') as f:
    for line in f:
        keylists.append(json.loads(line))

#Helper functions
def do_nothing(ex):
    return ex.lower()

def conllu_tokenizer(ex):
    return ex.replace("\n", "\t").replace("|", "\t").split("\t")

def whitespace_tokenizer(ex):
    return ex.split(" ")

/home/tenojo/miniconda3/envs/Test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def buildDatasetFromRawConllus(conllu_folder, age_mapping_file):
    to_convert = []
    isbn2age_series = pd.DataFrame(pd.read_excel(age_mapping_file, index_col=0))
    for key in os.listdir(conllu_folder):
        book_id = key[:14] +  str(isbn2age_series.at[int(key[:13]),isbn2age_series.columns[0]]) + key[15:17]
        age = isbn2age_series.at[int(key[:13]),isbn2age_series.columns[0]]
        with open(conllu_folder+key, 'r', encoding='UTF-8') as reader:
            conllu_text = reader.read()
        to_convert.append({'book_id':book_id, 'data':conllu_text})
    return Dataset.from_list(to_convert)

In [3]:
#Build / re-build the raw dataset
buildDatasetFromRawConllus('Conllus/', "ISBN_MAPS/ISBN2AGE.xlsx").save_to_disk("TCBC_datasets/RawDataset")

Saving the dataset (2/2 shards): 100%|██████████| 300/300 [00:00<00:00, 1529.51 examples/s]


In [4]:
#Function for generating a snippet dataset
def generateSnippetDatasetFromRawConllu(ex, sniplen):
        conllu = ex['data'][0]
        new_data = []
        df = cmf.snippetConllu2DF(conllu)
        tree = bdf.buildIdTreeFromConllu(df)
        df.loc[df['upos'] == 'PROPN', 'lemma'] = "[MASK]"
        df.loc[df['upos'] == 'PROPN', 'text'] = "[MASK]"
        conllu_lines = ["\t".join(x) for x in df.to_numpy("str").tolist()]
        tree_heads = list(tree.keys())
        start = 0
        for i in range(sniplen-1, len(tree), sniplen):
                new_data.append("\n".join(conllu_lines[start:list(tree[tree_heads[i]].keys())[-1]]))
                start = i
        return {"book_id":[ex['book_id'][0]]*len(new_data), "data": new_data}

In [5]:
#Generate and save to disk sniplen 100
#with_100 = Dataset.load_from_disk("TCBC_datasets/RawDataset").map(generateSnippetDatasetFromRawConllu, fn_kwargs={"sniplen":100}, batched=True, batch_size=1, remove_columns=['book_id', 'age', 'data'])

In [6]:
#Generate and save to disk sniplen 75
#Dataset.load_from_disk("TCBC_datasets/RawDataset").map(generateSnippetDatasetFromRawConllu, fn_kwargs={"sniplen":75}, batched=True, batch_size=1, remove_columns=['book_id', 'data']).save_to_disk("TCBC_datasets/sniplen75")

In [7]:
#Build / re-build the raw dataset
buildDatasetFromRawConllus('Conllus/', "ISBN_MAPS/ISBN2AGE.xlsx").save_to_disk("TCBC_datasets/RawDataset")

Saving the dataset (2/2 shards): 100%|██████████| 300/300 [00:00<00:00, 347.97 examples/s] 


In [8]:
#Generate and save to disk sniplen 50
#Dataset.load_from_disk("TCBC_datasets/sniplen100").map(generateSnippetDatasetFromRawConllu, fn_kwargs={"sniplen":50}, batched=True, batch_size=1, remove_columns=['book_id', 'data']).save_to_disk("TCBC_datasets/sniplen50")